# RNN Forward

Imports básicos

In [0]:
# Basic imports.
import numpy as np
import torch
from torch import nn

In [2]:
# Setting predefined arguments.
args = {}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

cuda


## Relembrando Redes Neurais Feed-Forward
Um perceptron comum, que integra uma Rede Neural Feed-Forward, realiza a seguinte operaçao:

\begin{align*}
f(X) = \sigma (W_{xh}X + b_h),
\end{align*}

sendo 
- $X \in R^{b \times d}$ um mini batch de $b$ amostras com $d$ dimensões;
- $W_{xh} \in R^{d \times h}$ a matriz de pesos, com $h$ sendo a dimensionalidade da saída desejada;
- $b_h \in R^{1 \times h}$ o viés;
- $\sigma$ a ativação não linear.

O resultado é $f(X) \in R^{b \times h}$, podendo tanto ser a feature intermediária que alimentará camadas futuras, ou a saída final do modelo. 

O código a seguir implementa um forward do zero de uma camada feed forward simples.  A nomenclatura *hidden_size* representa a dimensionalidade da saída da camada.

Hiperparâmetros
- batch size = 10
- input size = 100
- hidden size = 512
- função de ativação = tanh

In [3]:
def init_weights(input_size, hidden_size):
  params = {}
  params['Wxh'] = np.random.randn(input_size, hidden_size)
  params['bh']  = np.random.randn(1, hidden_size)
  
  return params

def forward(X, params):
  return np.tanh(np.dot(X, params['Wxh']) + params['bh'])

batch_size  = 10
input_size  = 100
hidden_size = 512

## Generate random batch 
X = np.random.randn(batch_size, input_size)

## Init weights
params = init_weights(input_size, hidden_size)

## Forward
output = forward(X, params)
print(output.shape)

(10, 512)


## Redes Recorrentes - *Forward from Scratch*

Uma rede neural recorrente, por outro lado, é uma função não só de $X$, mas também de um *hidden state* $H \in R^{b \times h}$, que representa a memória interna da unidade recorrente. Em outras palavras, a unidade recorrente é uma função da entrada atual $X_t$ e do conhecimento prévio acumulado $H_{t-1}$, com $t$ representando os timesteps. Para tal, adiciona-se uma matriz de pesos $W_{hh} \in R^{h \times h}$ que opera com a memória interna de timesteps anteriores, ou seja

\begin{align*}
f(X_t, H_{t-1}) = \sigma (W_{xh}X_t + W_{hh}H_{t-1} + b_h).
\end{align*}

Esse é um processo **iterativo**, que deve ser executado com todos os elementos $X_t$ da sequência de entrada $X = \{X_1, X_2,..., X_n\}$, sendo $n$ o tamanho da sequência. O forward produz o conjunto de *hidden states* $H = \{H_1, H_2, ..., H_n\}$.

**Atenção:** Na primeira iteração, o *hidden state* $H_0$ deve ser inicializado. Existem vários métodos de inicialização, sendo os mais comuns inicialização aleatória ou com zeros. Veja a seguir uma ilustração do processo iterativo na versão compacta (esquerda) e desenrolada (direita):

![](https://drive.google.com/uc?export=view&id=13lVkpXH5GqE8YjgpGyo8fFzRBKqPder7)

A célula a seguir contém o forward iterativo de uma camada recorrente simples. A saída apresenta é o $H \in R^{n \times b \times h}$ contendo todos os hidden states produzidos ao longo das iterações. 

**Atenção:** A entrada de uma unidade recorrente **não** segue mais o padrão $(B,C,H,W)$ que estávamos acostumados, seguindo agora o padrão $(N,B,D)$ sendo $N$ o tamanho da sequência e $D$ a dimensionalidade de cada elemento da sequência.

Hiperparâmetros
- **sequence length** = 50
- batch size = 10
- input size = 100
- hidden size = 512
- função de ativação = tanh

In [4]:
def init_weights(input_size, hidden_size):
  params = {}
  params['Wxh'] = np.random.randn(input_size, hidden_size)
  params['Whh'] = np.random.randn(hidden_size, hidden_size)
  params['bh']  = np.random.randn(1, hidden_size)
  
  return params

def forward(X, params):
  
  # Initialize hidden state for first iteration
  H = np.random.randn(batch_size, hidden_size)
  
  # Iterative forward
  hidden_states = []
  for x in X:
    H = np.tanh(np.dot(x, params['Wxh']) + np.dot(H, params['Whh']) + params['bh'] ) 
    hidden_states.append(H)
    
  return np.array(hidden_states) # output (seq_len, batch_size, hidden_size)

seq_len     = 50
batch_size  = 10
input_size  = 100
hidden_size = 512

## Generate random batch 
X = np.random.randn(seq_len, batch_size, input_size)

## Init weights
params = init_weights(input_size, hidden_size)

## Forward
output = forward(X, params)
print(output.shape)


(50, 10, 512)


## Pytorch RNNCell

Documentação: https://pytorch.org/docs/stable/nn.html#torch.nn.RNNCell

Essa camada realiza a operação de uma célula básica de RNN, que já conhecemos pela definição a seguir

\begin{align*}
H_t = f(X_t, H_{t-1}) = \sigma (W_{xh}X_t + W_{hh}H_{t-1} + b_h).
\end{align*}

O forward é realizado de forma idêntica ao que fizemos anteriormente, iterando na sequência para operar com cada um dos elementos. A saída a cada timestep $t$ é a memória atualizada $H_t$.

In [5]:
class RNN(nn.Module):
  
  def __init__(self, input_size, hidden_size, batch_size):
    super(RNN, self).__init__()
    
    self.hidden_size = hidden_size
    self.batch_size  = batch_size
    
    self.rnn = nn.RNNCell(input_size, hidden_size)

  def forward(self, X):

    # Initialize hidden state for first iteration
    H = torch.randn(self.batch_size, self.hidden_size).to(args['device'])

    # Iterative forward
    hidden_states = []
    for x in X:
      H = self.rnn(x, H)
      hidden_states.append(H)

    return torch.stack(hidden_states) # output (seq_len, batch_size, hidden_size)

net = RNN(input_size, hidden_size, batch_size).to(args['device'])
  
seq_len     = 50
batch_size  = 10
input_size  = 100
hidden_size = 512

## Generate random batch 
X = torch.randn(seq_len, batch_size, input_size).to(args['device'])

## Forward
output = net(X)
print(output.size())


torch.Size([50, 10, 512])
